In [ ]:
import scipy.integrate
import numpy as np
from numba import njit, vectorize, float32, float64, boolean
from numba.experimental import jitclass
from matplotlib import pyplot as plt

The problem is defined by the location of the currently active polymerases. Each polymerase is characterized by:
1. It's location
2. The length of the nascent mRNA tail ($x$)
3. The DNA twist at this location ($\phi$)
4. The RNAC at this location ($\theta$)

We actually only need to track the first three; the fourth is completely determined by the first three.

Given this state information, we can compute the relaxed supercoiling, which in turn gives us information on the 

In [ ]:
params = {
    'mRNA_drag': 1/20, # pN nm^(alpha / 1)
    'mRNA_exponent': 1, # the value of alpha
    'DNA_twist_mobility': 10, # s pN nm
    'RNAP_radius': 15, # nm
    'RNAP_velocity': 20, # nm / s
    'RNAP_torque_cutoff': 12, # pN nm
    'RNAP_torque_exponent': 100,
    'DNA_force': 1, # pN
    'DNA_bend_plength': 50, # pN
    'DNA_twist_plength': 95, # pN
    'DNA_plectonome_twist_plength': 24, # pN
    'temperature': 298 # K
}

In [ ]:
# Most member variables are float64's except for left_bc_free/right_bc_free
supercoiling_varspec = [(name,float64) for name in
                       ['kb', 'w0', 'tau_s', 'tau_0', 'tau_p', 'sc_s', 'sc_p',
                        'v_0', 't_c', 'torque_exp', 'chi', 'eta', 'alpha',
                       'left_bc_position', 'right_bc_position', 'left_bc_val', 'right_bc_val']]
supercoiling_varspec.append(('left_bc_free', boolean))
supercoiling_varspec.append(('right_bc_free', boolean))

@jitclass(supercoiling_varspec)
class SupercoilingPolymeraseModel(object):
    """
    Class that encapsulates all physical constants and
    calculations needed to compute an ODE model for
    supercoiling induced by transcription.
    
    It exposes the system_derivatives member function,
    which can be used in an ODE solver to represent the motion
    of each of the polymerases and the induced supercoiling.
    """
    def __init__(self, A, C, P, T, f,
                 v_0, critical_torque, torque_exponent,
                 chi, eta, alpha, left_bc_free, right_bc_free,
                 left_bc_loc, right_bc_loc, left_bc_val, right_bc_val):
        """
        Constructor that takes all relevant system constants and initalizes
        internal parameters used by other functions.
    
        Args:
        -----
        A: DNA bend persistence length (nm)
        C: DNA twist persistance length (nm)
        P: DNA plectonome twist persistence length (nm)
        T: Temperature of the system (K)
        f: Constant force held on DNA (pN)
        v_0: The maximum speed of the polymerase (nm / s)
        critical_torque: Torque (in pN nm) at which the
            polymerases begin to stall.
        torque_exponent: An exponent encoding how sharp the
            stalling torque is.
        chi: The DNA twisting mobility (s pN nm)
        eta: The The mRNA drag coefficent (pN (nm^(alpha - 1)))
        alpha: The mRNA power-law scaling exponent.
        left_bc_free: A boolean representing if the left BC is free.
        right_bc_free: A boolean representing if the right BC is free.
        left_bc_loc: If the left BC is not free, encodes the location of the BC.
        right_bc_loc: If the right BC is not free, encodes the location of the BC.
        left_bc_val: If the left BC is not free, encodes the value of phi at the BC.
        right_bc_val: If the right BC is not free, encodes the value of phi at the BC.
        """
        self.kb =  1380649 / 100000000 # pN nm / K
        self.w0 =  1.85 # 1/nm

        # Init torque model constants
        c = self.kb * T * C * self.w0 ** 2 # pN
        p = self.kb * T * P * self.w0 ** 2 # pN
        g = f - np.sqrt(self.kb * T * f / A) # pN
        cs = c * (1 - C / (4 * A) * np.sqrt(self.kb * T / (A * f))) # pN

        # Compute critical torque model values
        self.tau_s = cs / self.w0 # pN nm
        self.tau_0 = np.sqrt(2 * p * g / (self.w0 ** 2 * (1 - p / cs))) # pN nm
        self.tau_p = p / self.w0 # pN nm
        self.sc_s = 1 / cs * np.sqrt(2 * p * g  / (1 - p / cs)) # dimless
        self.sc_p = 1 / p * np.sqrt(2 * p * g / (1 - p / cs)) # dimless
        
        self.v_0 = v_0
        self.t_c = critical_torque
        self.torque_exp = torque_exponent
        self.chi = chi
        self.eta = eta
        self.alpha = alpha
        self.left_bc_free = left_bc_free
        self.right_bc_free = right_bc_free
        self.left_bc_position = left_bc_loc
        self.right_bc_position = right_bc_loc
        self.left_bc_val = left_bc_val
        self.right_bc_val = right_bc_val
    
    def torque_response(self, sc):
        """
        Given a ndarray of supercoiling values, calculates
        the torque response on each using vectorized Numpy functions.
        
        The torque response is a function of the provided
        DNA persistence lengths, the temperature and force,
    
        Args:
        -----
        sc: A ndarray containing supercoiling density values.
        
        Returns:
        --------
        The calculated torque, using a two-phase constant-force
        torque equation.
        """
        abs_sc = np.abs(sc)
        result = self.tau_0 * np.sign(sc)
        result[abs_sc < self.sc_s] = self.tau_s * sc
        result[abs_sc >= self.sc_p] = self.tau_p * sc
        return result
    
    def polymerase_velocity(self, sc_behind, sc_ahead):
        """
        Given supercoiling densities behind and ahead of
        each polymerase, returns the velocity of each
        polymerase.
        
        Args:
        -----
        sc_behind: An ndarray containing the supercoiling
            density behind each polymerase.
        sc_ahead: An ndarray containing the supercoiling
            density ahead of each polymerase.
        
        Returns:
        --------
        The velocity of each polymerase (in nm/s).
        """
        return self.v_0  / (
               (1 + np.abs(
                   self.torque_response(sc_behind) / self.t_c)**self.torque_exp) *
               (1 + np.abs(
                   self.torque_response(sc_ahead) / self.t_c)**self.torque_exp))
    
    def system_derivatives(self, states, polymerase_directions):
        """
        Given the state of the system, in terms of a location, nascant mRNA length,
        and local excess DNA rotation, computes the derivatives of all of these states.
    
        Args:
        -----
        states: A (3N,) shape ndarray encoding (location, mRNA, phi) for each polymerase.
            This vector is assumed to be sorted in increasing order of location
        polymerase_directions: A (N,) shape ndarray encoding +-1, depending on the
            direction that the polymerase is moving.

        Returns:
        --------
        A 3N-length vector encoding the time derivatives of each of the states.
        """
        augmented_loc = np.concatenate((np.array([self.left_bc_position]),
                                        states[::3],
                                        np.array([self.right_bc_position])))
        augmented_phi = np.concatenate((np.array([self.left_bc_val]),
                                        states[2::3],
                                        np.array([self.right_bc_val])))
        # Replace these with free BCs if needed
        if self.left_bc_free:
            # Step w0 away from BC, to avoid divide by zero errors.
            augmented_loc[0] = augmented_loc[1] - 1
            augmented_phi[0] = augmented_phi[1]
        if self.right_bc_free:
            augmented_loc[-1] = augmented_loc[-2] + 1
            augmented_phi[-1] = augmented_phi[-2]
        # Compute supercoiling in each N + 1 region
        supercoiling = np.diff(augmented_phi) / (np.diff(augmented_loc) * -self.w0)
        rnac_torque = self.torque_response(supercoiling[1:]) - self.torque_response(supercoiling[:-1])
        velocities = polymerase_directions * self.polymerase_velocity(supercoiling[1:], supercoiling[:-1])
        drag = self.eta * (states[1::3] ** self.alpha)
        angular_changes =  drag * velocities * self.w0 / (self.chi + drag) - rnac_torque / (self.chi + drag)

        derivatives = np.zeros(states.shape)
        derivatives[::3] = velocities
        derivatives[1::3] = velocities
        derivatives[2::3] = angular_changes
        return derivatives

def bind_supercoiling_model(params, bcs):
    """
    Given the system parameters and boundary conditions,
    returns a PolymeraseSupercoilingModel representing the system.
    
    Args:
    -----
    params: A dictionary containing the following keys:
        mRNA_drag:            pN nm^(alpha / 1)
        mRNA_exponent:        the value of alpha
        DNA_twist_mobility:   s pN nm
        RNAP_radius:           nm
        RNAP_velocity:        nm / s
        RNAP_torque_cutoff:   pN nm
        RNAP_torque_exponent: dimless
        DNA_force:            pN
        DNA_bend_plength:     pN
        DNA_twist_plength:    pN
        DNA_plectonome_twist_plength: pN
        temperature:          K
    bcs: A 2-tuple encoding (left_bc, right_bc). A BC is either
        the string "free", or a tuple of (location, value).
    
    Returns:
    --------
    A SupercoilingPolymeraseModel with all relevant values set.
    """
    # Calculate BCs
    left_bc_free = bcs[0] == 'free'
    right_bc_free = bcs[1] == 'free'
    left_bc = (0, 0)
    right_bc = (0, 0)
    if not left_bc_free:
        left_bc = bcs[0]
    if not right_bc_free:
        right_bc = bcs[1]
    
    return SupercoilingPolymeraseModel(
            params['DNA_bend_plength'], params['DNA_twist_plength'],
            params['DNA_plectonome_twist_plength'], params['temperature'],
            params['DNA_force'], params['RNAP_velocity'],
            params['RNAP_torque_cutoff'], params['RNAP_torque_exponent'],
            params['DNA_twist_mobility'], params['mRNA_drag'],
            params['mRNA_exponent'], left_bc_free, right_bc_free,
            left_bc[0], right_bc[0], left_bc[1], right_bc[1])

TODO: implement RNA hard-sphere repulsion radius

In [ ]:
sys_derivatives = bind_system_derivatives(params, ('free', (1000, 0)))
# Setup a system with a single polymerase

result = scipy.integrate.solve_ivp(lambda t,y: sys_derivatives(y, [1]), (0, 2000), ics, method='Radau')

In [ ]:
model = bind_supercoiling_model(params, ('free', (1000,0)))
ics = np.array([0, 0, 0])
result = scipy.integrate.solve_ivp(lambda t,y: model.system_derivatives(y,np.array([1.0])), (0,2000), ics)

In [ ]:
%timeit model.system_derivatives(ics,np.array([1.0]))

In [ ]:

plt.plot(result.t, result.y.T)
plt.legend(('RNAP location', 'Nascent mRNA length', 'Excess DNA twist'))
plt.show()
plt.plot(result.t, result.y[2,:])
plt.show()

plt.plot(test_result.t, test_result.y.T)
plt.legend(('RNAP location', 'Nascent mRNA length', 'Excess DNA twist'))
plt.show()
plt.plot(test_result.t, test_result.y[2,:])

In [ ]:
@vectorize(signatures="float64(float64)")
def test_ufunc(x):
    return 2 * x;
test_ufunc.ufunc